## Prerequisites

In [1]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets
import os
import glob

pd.set_option("display.max_colwidth", None)

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument
from transformers import AutoTokenizer
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from ragatouille import RAGPretrainedModel
from langchain_core.vectorstores import VectorStore
from langchain_core.language_models.llms import LLM
from langchain_community.llms import HuggingFaceHub
from langchain_community.document_loaders import PyPDFLoader

In [3]:
# Import the load_dotenv function from the dotenv module
from dotenv import load_dotenv

# Call the load_dotenv function to load environment variables from a .env file
load_dotenv()

os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [4]:
class RAG_pipeline:

    def __init__(self, data_dir_path: str, chunk_size: int):
        self.data_dir_path = data_dir_path
        self.load_documents(self.data_dir_path)
        self.chunk_size = chunk_size
        self.RAG_PROMPT_TEMPLATE = """
            <|system|>
            Using the information contained in the context,
            give a comprehensive answer to the question.
            Respond only to the question asked, response should be concise and relevant to the question.
            Provide the number of the source document when relevant.
            If the answer cannot be deduced from the context, do not give an answer.</s>
            <|user|>
            Context:
            {context}
            ---
            Now here is the question you need to answer.

            Question: {question}
            </s>
            <|assistant|>
        """
    
    def load_documents(self, data_dir_path: str):
        docs = []
        for file_path in glob.glob(data_dir_path + "/*.pdf"):
            loader = PyPDFLoader(file_path)
            pages = loader.load_and_split()
            docs.extend(pages)

        self.knowledge_base = [
            LangchainDocument(page_content=page.page_content, metadata=page.metadata) for page in tqdm(docs)]

    def split_documents(self, tokenizer_name: str) -> List[LangchainDocument]:
        """
        Split documents into chunks of size `chunk_size` characters and return a list of documents.
        """
        text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
            AutoTokenizer.from_pretrained(tokenizer_name),
            chunk_size=self.chunk_size,
            chunk_overlap=int(self.chunk_size / 10),
            add_start_index=True,
            strip_whitespace=True,
            separators=["\n\n", "\n", ".", " ", ""],
        )

        docs_processed = []
        for doc in self.knowledge_base:
            docs_processed += text_splitter.split_documents([doc])

        # Remove duplicates
        unique_texts = {}
        docs_processed_unique = []
        for doc in docs_processed:
            if doc.page_content not in unique_texts:
                unique_texts[doc.page_content] = True
                docs_processed_unique.append(doc)
    
        return docs_processed_unique
    
    def load_embeddings(self,
        embedding_model_name: Optional[str] = "thenlper/gte-small") -> FAISS:
        """
        Creates a FAISS index from the given embedding model and documents. Loads the index directly if it already exists.

        Args:
            langchain_docs: list of documents
            chunk_size: size of the chunks to split the documents into
            embedding_model_name: name of the embedding model to use

        Returns:
            FAISS index
        """
        # load embedding_model
        embedding_model = HuggingFaceEmbeddings(
            model_name=embedding_model_name,
            multi_process=True,
            model_kwargs={"device": "cpu"},
            encode_kwargs={"normalize_embeddings": True},  # set True to compute cosine similarity
        )

        # Check if embeddings already exist on disk
        index_name = f"index_chunk:{self.chunk_size}_embeddings:{embedding_model_name.replace('/', '~')}"
        index_folder_path = f"./data/indexes/{index_name}/"
        if os.path.isdir(index_folder_path):
            return FAISS.load_local(
                index_folder_path,
                embedding_model,
                distance_strategy=DistanceStrategy.COSINE,
                allow_dangerous_deserialization=True
            )

        else:
            print("Index not found, generating it...")
            docs_processed = self.split_documents(
                embedding_model_name,
            )
            knowledge_index = FAISS.from_documents(
                docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
            )
            knowledge_index.save_local(index_folder_path)
            return knowledge_index
        
    def answer_with_rag(self, question: str,
        llm: LLM,
        knowledge_index: VectorStore,
        reranker: Optional[RAGPretrainedModel] = None,
        num_retrieved_docs: int = 30,
        num_docs_final: int = 7) -> Tuple[str, List[LangchainDocument]]:
        """Answer a question using RAG with the given knowledge index."""
        # Gather documents with retriever
        relevant_docs = knowledge_index.similarity_search(query=question, k=num_retrieved_docs)
        relevant_docs = [doc.page_content for doc in relevant_docs]  # keep only the text

        # Optionally rerank results
        if reranker:
            relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
            relevant_docs = [doc["content"] for doc in relevant_docs]

        relevant_docs = relevant_docs[:num_docs_final]

        # Build the final prompt
        context = "\nExtracted documents:\n"
        context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)])

        final_prompt = self.RAG_PROMPT_TEMPLATE.format(question=question, context=context)

        # Redact an answer
        answer = llm.invoke(final_prompt)

        return answer, relevant_docs

In [5]:
rag_pipeline = RAG_pipeline(data_dir_path="./data", chunk_size=100)

  0%|          | 0/662 [00:00<?, ?it/s]

In [6]:
index = rag_pipeline.load_embeddings()

In [7]:
from langchain_community.llms import HuggingFaceHub

repo_id = "HuggingFaceH4/zephyr-7b-beta"
READER_MODEL_NAME = "zephyr-7b-beta"

READER_LLM = HuggingFaceHub(
    repo_id=repo_id,
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

/Users/priyanshutuli/Desktop/RAG_pipeline_testing/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [65]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch
import accelerate

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer=AutoTokenizer.from_pretrained(model)
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="cpu",
    max_length=1000,
    do_sample=True,
    # num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [66]:
llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={"max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03})

In [39]:
repo_id = "HuggingFaceH4/zephyr-7b-beta"
READER_MODEL_NAME = "zephyr-7b-beta"

READER_LLM = HuggingFaceHub(
    repo_id=repo_id,
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

In [63]:
# repo_id = "mistralai/Mistral-7B-Instruct-v0.1"
# READER_MODEL_NAME = "Mistral-7B-Instruct-v0.1"

# LLM_CHAT = HuggingFaceHub(
#     repo_id=repo_id,
#     task="text-generation",
#     model_kwargs={
#         "max_new_tokens": 512,
#         "top_k": 30,
#         "temperature": 0.1,
#         "repetition_penalty": 1.03,
#     },
# )

In [40]:
question = "What is CVA estimated based on?"

In [41]:
# READER_LLM = llm

In [42]:
answer, relevant_docs = rag_pipeline.answer_with_rag(question = question, llm = READER_LLM, knowledge_index=index, num_docs_final=4)

In [43]:
answer

'\n            <|system|>\n            Using the information contained in the context,\n            give a comprehensive answer to the question.\n            Respond only to the question asked, response should be concise and relevant to the question.\n            Provide the number of the source document when relevant.\n            If the answer cannot be deduced from the context, do not give an answer.</s>\n            <|user|>\n            Context:\n            \nExtracted documents:\nDocument 0:::\nKey assumptions:\nPrepayment rate assumption (1)  8.5%  9.4 \nImpact on fair value from 10% adverse change $ 244  288 \nImpact on fair value from 25% adverse change  586  688 \nDiscount rate assumption  9.6%  9.1 \nImpact on fair value from 100 basis point increase $ 340  368Document 1:::\nImpact on fair value from 200 basis point increase  652  707 \nCost to service assumption ($ per loan)  102  102 \nImpact on fair value from 10% adverse change  157  171 \nImpact on fair value from 25% 

In [44]:
relevant_docs

['Key assumptions:\nPrepayment rate assumption (1)  8.5%  9.4 \nImpact on fair value from 10% adverse change $ 244  288 \nImpact on fair value from 25% adverse change  586  688 \nDiscount rate assumption  9.6%  9.1 \nImpact on fair value from 100 basis point increase $ 340  368',
 'Impact on fair value from 200 basis point increase  652  707 \nCost to service assumption ($ per loan)  102  102 \nImpact on fair value from 10% adverse change  157  171 \nImpact on fair value from 25% adverse change  391  427',
 'Impact on fair value from 25% adverse change 684 688 \nDiscount rate assumption 8.9 % 9.1 \nImpact on fair value from 100 basis point increase $ 346 368 \nImpact on fair value from 200 basis point increase 664 707 \nCost to service assumption ($ per loan) 101 102 \nImpact on fair value from 10% adverse change 166 171',
 'Impact on fair value from 100 basis point increase $ 316 368 \nImpact on fair value from 200 basis point increase 608 707 \nCost to service assumption ($ per loan)

In [17]:
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [14]:
from trulens_eval import TruChain, Tru
tru = Tru()
tru.reset_database()
from trulens_eval.feedback.provider import Langchain
from trulens_eval import Feedback

/Users/priyanshutuli/Desktop/RAG_pipeline_testing/.venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1007)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1007)>


🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [16]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [18]:
retriever = index.as_retriever()

In [19]:
from langchain import hub

In [21]:
prompt = hub.pull("rlm/rag-prompt")

In [45]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | READER_LLM
    | StrOutputParser()
)

In [46]:
rag_chain.invoke("What is CVA estimated based on?")

"Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: What is CVA estimated based on? \nContext: Key assumptions:\nPrepayment rate assumption (1)  8.5%  9.4 \nImpact on fair value from 10% adverse change $ 244  288 \nImpact on fair value from 25% adverse change  586  688 \nDiscount rate assumption  9.6%  9.1 \nImpact on fair value from 100 basis point increase $ 340  368\n\nImpact on fair value from 200 basis point increase  652  707 \nCost to service assumption ($ per loan)  102  102 \nImpact on fair value from 10% adverse change  157  171 \nImpact on fair value from 25% adverse change  391  427\n\nImpact on fair value from 25% adverse change 684 688 \nDiscount rate assumption 8.9 % 9.1 \nImpact on fair value from 100 basis point increase $ 346 368 \nImpact on fair value from 200 basis

In [25]:
from trulens_eval.feedback.provider.hugs import Huggingface
huggingface_provider = Huggingface()

In [32]:
from trulens_eval.feedback.provider import Langchain

In [80]:
from llama_cpp import Llama

In [83]:
llm = Llama.from_pretrained(
    repo_id="Qwen/Qwen1.5-0.5B-Chat-GGUF",
    filename="*q8_0.gguf",
    verbose=True,
    local_dir="./models"
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from ./models/qwen1_5-0_5b-chat-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.name str              = Qwen1.5-0.5B-Chat-AWQ-fp16
llama_model_loader: - kv   2:                          qwen2.block_count u32              = 24
llama_model_loader: - kv   3:                       qwen2.context_length u32              = 32768
llama_model_loader: - kv   4:                     qwen2.embedding_length u32              = 1024
llama_model_loader: - kv   5:                  qwen2.feed_forward_length u32              = 2816
llama_model_loader: - kv   6:                 qwen2.attention.head_count u32              = 16
llama_model_loader: - kv   7:              qwen2.attentio

In [84]:
from langchain_community.llms import LlamaCpp

llama = LlamaCpp(model_path = "/Users/priyanshutuli/Desktop/RAG_pipeline_testing/models/qwen1_5-0_5b-chat-q8_0.gguf")

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /Users/priyanshutuli/Desktop/RAG_pipeline_testing/models/qwen1_5-0_5b-chat-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.name str              = Qwen1.5-0.5B-Chat-AWQ-fp16
llama_model_loader: - kv   2:                          qwen2.block_count u32              = 24
llama_model_loader: - kv   3:                       qwen2.context_length u32              = 32768
llama_model_loader: - kv   4:                     qwen2.embedding_length u32              = 1024
llama_model_loader: - kv   5:                  qwen2.feed_forward_length u32              = 2816
llama_model_loader: - kv   6:                 qwen2.attention.head_count u32              = 16
llama_mod

In [85]:
lc = Langchain(llama)

In [27]:
from trulens_eval.app import App
context = App.select_context(rag_chain)

In [68]:
from trulens_eval.feedback import Groundedness
import numpy as np
grounded = Groundedness(groundedness_provider=huggingface_provider)
# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons)
    .on(context.collect()) # collect context chunks into a list
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(lc.relevance)
    .on_input_output()
)
# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(lc.context_relevance_with_cot_reasons)
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

✅ In groundedness_measure_with_cot_reasons, input source will be set to __record__.app.first.steps.context.first.get_relevant_documents.rets.collect() .
✅ In groundedness_measure_with_cot_reasons, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In context_relevance_with_cot_reasons, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In context_relevance_with_cot_reasons, input context will be set to __record__.app.first.steps.context.first.get_relevant_documents.rets .


[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1007)>


In [87]:
tru_recorder = TruChain(rag_chain,
    app_id='Chain1_ChatApplication',
    feedbacks=[f_answer_relevance, f_context_relevance, f_groundedness])

In [88]:
response, tru_record = tru_recorder.with_record(rag_chain.invoke, "What is CVA estimated based on?")

In [89]:
response

"Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: What is CVA estimated based on? \nContext: Key assumptions:\nPrepayment rate assumption (1)  8.5%  9.4 \nImpact on fair value from 10% adverse change $ 244  288 \nImpact on fair value from 25% adverse change  586  688 \nDiscount rate assumption  9.6%  9.1 \nImpact on fair value from 100 basis point increase $ 340  368\n\nImpact on fair value from 200 basis point increase  652  707 \nCost to service assumption ($ per loan)  102  102 \nImpact on fair value from 10% adverse change  157  171 \nImpact on fair value from 25% adverse change  391  427\n\nImpact on fair value from 25% adverse change 684 688 \nDiscount rate assumption 8.9 % 9.1 \nImpact on fair value from 100 basis point increase $ 346 368 \nImpact on fair value from 200 basis

In [90]:
tru_record

Record(record_id='record_hash_66b34c52dca741c19f2ee6b2ee5668de', app_id='Chain1_ChatApplication', cost=Cost(n_requests=0, n_successful_requests=0, n_classes=0, n_tokens=0, n_stream_chunks=0, n_prompt_tokens=0, n_completion_tokens=0, cost=0.0), perf=Perf(start_time=datetime.datetime(2024, 4, 15, 20, 2, 56, 19085), end_time=datetime.datetime(2024, 4, 15, 20, 3, 7, 102234)), ts=datetime.datetime(2024, 4, 15, 20, 3, 7, 102298), tags='-', meta=None, main_input='What is CVA estimated based on?', main_output="Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: What is CVA estimated based on? \nContext: Key assumptions:\nPrepayment rate assumption (1)  8.5%  9.4 \nImpact on fair value from 10% adverse change $ 244  288 \nImpact on fair value from 25% adverse change  586  688 \nDiscount rate as

In [50]:
json_like = tru_record.layout_calls_as_app()

In [52]:
from ipytree import Tree, Node

def display_call_stack(data):
    tree = Tree()
    tree.add_node(Node('Record ID: {}'.format(data['record_id'])))
    tree.add_node(Node('App ID: {}'.format(data['app_id'])))
    tree.add_node(Node('Cost: {}'.format(data['cost'])))
    tree.add_node(Node('Performance: {}'.format(data['perf'])))
    tree.add_node(Node('Timestamp: {}'.format(data['ts'])))
    tree.add_node(Node('Tags: {}'.format(data['tags'])))
    tree.add_node(Node('Main Input: {}'.format(data['main_input'])))
    tree.add_node(Node('Main Output: {}'.format(data['main_output'])))
    tree.add_node(Node('Main Error: {}'.format(data['main_error'])))
    
    calls_node = Node('Calls')
    tree.add_node(calls_node)
    
    for call in data['calls']:
        call_node = Node('Call')
        calls_node.add_node(call_node)
        
        for step in call['stack']:
            step_node = Node('Step: {}'.format(step['path']))
            call_node.add_node(step_node)
            if 'expanded' in step:
                expanded_node = Node('Expanded')
                step_node.add_node(expanded_node)
                for expanded_step in step['expanded']:
                    expanded_step_node = Node('Step: {}'.format(expanded_step['path']))
                    expanded_node.add_node(expanded_step_node)
    
    return tree

# Usage
tree = display_call_stack(json_like)
tree

Tree(nodes=(Node(name='Record ID: record_hash_fc2c83d61cb4f0a5c0a28de731764f45'), Node(name='App ID: Chain1_Ch…

In [72]:
with tru_recorder as recording:
    llm_response = rag_chain.invoke("What is CVA estimated based on?")

display(llm_response)

"Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: What is CVA estimated based on? \nContext: Key assumptions:\nPrepayment rate assumption (1)  8.5%  9.4 \nImpact on fair value from 10% adverse change $ 244  288 \nImpact on fair value from 25% adverse change  586  688 \nDiscount rate assumption  9.6%  9.1 \nImpact on fair value from 100 basis point increase $ 340  368\n\nImpact on fair value from 200 basis point increase  652  707 \nCost to service assumption ($ per loan)  102  102 \nImpact on fair value from 10% adverse change  157  171 \nImpact on fair value from 25% adverse change  391  427\n\nImpact on fair value from 25% adverse change 684 688 \nDiscount rate assumption 8.9 % 9.1 \nImpact on fair value from 100 basis point increase $ 346 368 \nImpact on fair value from 200 basis

In [91]:
# The record of the app invocation can be retrieved from the `recording`:

rec = recording.get() # use .get if only one record
# recs = recording.records # use .records if multiple

display(rec)

Record(record_id='record_hash_70e192b2f9cdc3b11eada9a778231f9d', app_id='Chain1_ChatApplication', cost=Cost(n_requests=0, n_successful_requests=0, n_classes=0, n_tokens=0, n_stream_chunks=0, n_prompt_tokens=0, n_completion_tokens=0, cost=0.0), perf=Perf(start_time=datetime.datetime(2024, 4, 15, 19, 43, 5, 431941), end_time=datetime.datetime(2024, 4, 15, 19, 43, 14, 933133)), ts=datetime.datetime(2024, 4, 15, 19, 43, 14, 933200), tags='-', meta=None, main_input='What is CVA estimated based on?', main_output="Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: What is CVA estimated based on? \nContext: Key assumptions:\nPrepayment rate assumption (1)  8.5%  9.4 \nImpact on fair value from 10% adverse change $ 244  288 \nImpact on fair value from 25% adverse change  586  688 \nDiscount ra

In [92]:
# The results of the feedback functions can be rertireved from
# `Record.feedback_results` or using the `wait_for_feedback_result` method. The
# results if retrieved directly are `Future` instances (see
# `concurrent.futures`). You can use `as_completed` to wait until they have
# finished evaluating or use the utility method:

for feedback, feedback_result in rec.wait_for_feedback_results().items():
    print(feedback.name, feedback_result.result)

# See more about wait_for_feedback_results:
# help(rec.wait_for_feedback_results)

relevance None
context_relevance_with_cot_reasons None
groundedness_measure_with_cot_reasons None


In [56]:
records, feedback = tru.get_records_and_feedback(app_ids=["Chain1_ChatApplication"])

records.head()

app_id  \
0  Chain1_ChatApplication   
1  Chain1_ChatApplication   
2  Chain1_ChatApplication   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [57]:
tru.get_leaderboard(app_ids=["Chain1_ChatApplication"])

,latency,total_cost
app_id,,
Chain1_ChatApplication,11.333333,0.0


In [60]:
tru.stop_dashboard() # stop if needed

RuntimeError: Dashboard not running in this workspace. You may be able to shut other instances by setting the `force` flag.

In [62]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: None


<Popen: returncode: 1 args: ['streamlit', 'run', '--server.headless=True', '...>